In [1]:
import pickle

lm = pickle.load(open('lm.pkl', 'rb'))
transformer = pickle.load(open('transformer.pkl', 'rb'))
encoder = pickle.load(open('encoder.pkl', 'rb'))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import math

In [3]:
df = pd.read_csv('fifa21_validate.csv')
#print(df.shape)
#print(df.dtypes)
print(df.isna().sum())
pd.set_option('display.max_columns', None)
print(df.shape)

ID             0
Name           0
Age            0
Nationality    0
Club           3
              ..
CB             0
RCB            0
RB             0
GK             0
OVA            0
Length: 101, dtype: int64
(1999, 101)


In [4]:
def datacleaner(df):
    
    df['Club'] = df['Club'].fillna('Free Agent')
    df = df.drop(['Age','Nationality','Club','Team & Contract','Height','Weight','foot','Growth','Joined','Loan Date End','Value','Wage','Release Clause','Contract','W/F','SM','A/W','D/W','Hits','Position','ID','Name'], axis=1)
    df['IR'] = df['IR'].str.replace(" ★", "")
    def remove_plus_and_after(value):
         if isinstance(value, str):
            return value.split('+')[0]
         return value
    df = df.applymap(remove_plus_and_after)
    cols = ['IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'LS','ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM','LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB','LCB', 'CB', 'RCB', 'RB', 'GK']
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    column_means = df.mean()
    df = df.fillna(column_means)
    
    return df

modified_df = datacleaner(df)

/var/folders/kd/gp0d9cx1091g1l19n5h72dhr0000gn/T/ipykernel_14074/959859787.py:13: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


In [5]:
modified_df.isna().sum()

BP                  0
Attacking           0
Crossing            0
Finishing           0
Heading Accuracy    0
                   ..
CB                  0
RCB                 0
RB                  0
GK                  0
OVA                 0
Length: 79, dtype: int64

In [6]:
modified_df

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,IR,PAC,SHO,PAS,DRI,DEF,PHY,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CB,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,1,64,30,50,50,66,74,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67
1,CAM,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,1,74,67,62,74,24,57,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68
2,GK,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,1,56,52,53,55,26,53,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54
3,CDM,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,1,58,34,47,52,53,59,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55
4,CDM,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,1,67,62,68,70,69,71,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,LB,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,1,76,28,46,55,53,57,44,44,44,51,48,48,48,51,48,48,48,53,48,48,48,53,58,52,52,52,58,58,54,54,54,58,15,60
1995,GK,77,13,9,14,34,7.0,70,9,13.0,10,25,13,192,31,30,36.0,59,36.0,177,43,44.0,23,60,7,88,21,8.0,4.0,36.0,19,33.0,36,14,11,11.0,288,60,55,57,54,62,928,318,1,60,55,57,62,30,54,22,22,22,20,22,22,22,20,23,23,23,22,24,24,24,22,20,23,23,23,20,20,22,22,22,20,58,59
1996,RM,332,76,72,34,79,71.0,373,77,76.0,65,77,78,351,64,65,74.0,73,75.0,329,63,60.0,83,57,66,306,41,40.0,75.0,81.0,69,75.0,118,36,43,39.0,58,12,15,16,9,6,1867,388,2,65,69,78,77,39,60,67,67,67,74,73,73,73,74,76,76,76,75,74,74,74,75,62,60,60,60,62,57,47,47,47,57,18,76
1997,CDM,268,58,44,61,62,43.0,261,56,47.0,37,60,61,288,54,52,57.0,61,64.0,300,53,65.0,68,68,46,297,74,62.0,54.0,56.0,51,60.0,182,61,62,59.0,43,10,7,9,9,8,1639,346,1,53,47,58,58,61,69,55,55,55,56,55,55,55,56,57,57,57,57,59,59,59,57,60,62,62,62,60,60,63,63,63,60,14,63


In [7]:
X = modified_df.drop(['OVA'], axis=1)
y2 = modified_df['OVA']
X_cat = modified_df.select_dtypes('object')
X_num = X._get_numeric_data()
from sklearn.preprocessing import OneHotEncoder
cols = encoder.get_feature_names_out(input_features=X_cat.columns)
X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
X_num_normal = pd.DataFrame(x_normalized, columns=X_num.columns)
X = pd.concat([X_cat_encode, X_num_normal], axis=1)
    

(1999, 77)


In [8]:
X

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,BP_RM,BP_RW,BP_RWB,BP_ST,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,IR,PAC,SHO,PAS,DRI,DEF,PHY,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.487624,0.465909,0.195652,0.647727,0.604651,0.418605,0.433255,0.428571,0.433333,0.348315,0.500000,0.538462,0.541311,0.559524,0.670588,0.597561,0.541667,0.4000,0.498392,0.433735,0.742857,0.458824,0.870130,0.233333,0.532033,0.678161,0.633333,0.382979,0.341176,0.360465,0.440476,0.732,0.730337,0.726190,0.702381,0.103044,0.112360,0.069767,0.151163,0.120879,0.089888,0.521297,0.404580,0.00,0.542857,0.171053,0.367647,0.328358,0.701299,0.712121,0.434211,0.434211,0.434211,0.435897,0.410256,0.410256,0.410256,0.435897,0.415584,0.415584,0.415584,0.460526,0.479452,0.479452,0.479452,0.460526,0.625000,0.642857,0.642857,0.642857,0.625000,0.681159,0.708333,0.708333,0.708333,0.681159,0.088608
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.655941,0.636364,0.619565,0.443182,0.627907,0.767442,0.672131,0.780220,0.711111,0.573034,0.607143,0.714286,0.746439,0.761905,0.717647,0.841463,0.555556,0.7750,0.610932,0.771084,0.528571,0.600000,0.350649,0.788889,0.643454,0.724138,0.233333,0.659574,0.635294,0.604651,0.630952,0.176,0.179775,0.202381,0.154762,0.096019,0.146067,0.127907,0.081395,0.076923,0.078652,0.632549,0.496183,0.00,0.685714,0.657895,0.544118,0.686567,0.155844,0.454545,0.644737,0.644737,0.644737,0.692308,0.679487,0.679487,0.679487,0.692308,0.675325,0.675325,0.675325,0.684211,0.616438,0.616438,0.616438,0.684211,0.486111,0.442857,0.442857,0.442857,0.486111,0.449275,0.305556,0.305556,0.305556,0.449275,0.101266
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037129,0.011364,0.021739,0.068182,0.151163,0.000000,0.021077,0.010989,0.044444,0.033708,0.130952,0.054945,0.148148,0.178571,0.164706,0.231707,0.236111,0.2625,0.122186,0.337349,0.342857,0.117647,0.441558,0.022222,0.058496,0.126437,0.044444,0.053191,0.188235,0.069767,0.226190,0.040,0.056180,0.035714,0.047619,0.601874,0.617978,0.581395,0.593023,0.560440,0.606742,0.048951,0.255725,0.00,0.428571,0.460526,0.411765,0.402985,0.181818,0.393939,0.039474,0.039474,0.039474,0.012821,0.025641,0.025641,0.025641,0.012821,0.012987,0.012987,0.012987,0.013158,0.027397,0.027397,0.027397,0.013158,0.027778,0.028571,0.028571,0.028571,0.027778,0.028986,0.027778,0.027778,0.027778,0.028986,0.556962
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.450495,0.363636,0.304348,0.568182,0.593023,0.325581,0.423888,0.505495,0.333333,0.370787,0.452381,0.538462,0.461538,0.547619,0.552941,0.512195,0.333333,0.4625,0.350482,0.433735,0.328571,0.564706,0.493506,0.266667,0.487465,0.597701,0.555556,0.329787,0.352941,0.383721,0.559524,0.568,0.561798,0.547619,0.571429,0.056206,0.067416,0.034884,0.127907,0.032967,0.056180,0.413223,0.286260,0.00,0.457143,0.223684,0.323529,0.358209,0.532468,0.484848,0.407895,0.407895,0.407895,0.423077,0.397436,0.397436,0.397436,0.423077,0.402597,0.402597,0.402597,0.447368,0.452055,0.452055,0.452055,0.447368,0.541667,0.542857,0.542857,0.542857,0.541667,0.565217,0.527778,0.527778,0.527778,0.565217,0.025316
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.648515,0.579545,0.608696,0.454545,0.813953,0.604651,0.665105,0.725275,0.588889,0.516854,0.773810,0.758242,0.589744,0.654762,0.647059,0.634146,0.555556,0.4875,0.655949,0.734940,0.442857,0.729412,0.675325,0.644444,0.721448,0.655172,0.811111,0.542553,0.611765,0.546512,0.607143,0.776,0.775281,0.738095,0.773

In [9]:
y2

0       67
1       68
2       54
3       55
4       70
        ..
1994    60
1995    59
1996    76
1997    63
1998    60
Name: OVA, Length: 1999, dtype: int64

In [12]:
X_predict = lm.predict(X)

In [13]:
X_predict

array([65.18479156, 65.97034454, 53.32749176, ..., 73.75579071,
       63.2255249 , 62.19729614])

In [14]:
y2

0       67
1       68
2       54
3       55
4       70
        ..
1994    60
1995    59
1996    76
1997    63
1998    60
Name: OVA, Length: 1999, dtype: int64

In [11]:
r2_score(y2, X_predict)

-304026097468470.3